Imports

In [4]:
import psycopg2
from config import config

Define connect method to connect to the postgres database and return the connection

In [9]:
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        return conn

        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

Connect to the database

In [16]:
conn = connect()

Connecting to the PostgreSQL database...


Create the database tables

In [41]:
cur = conn.cursor()

# Country Table
cur.execute("""CREATE TABLE Country (
                Country_key int PRIMARY KEY,
                Name varchar(255),
                Region varchar(255),
                Continent varchar(255),
                Currency varchar(255),
                Capital varchar(255),
                Total_population int,
                Birth_rate float,
                Gross_national_income float,
                Life_expectancy_at_birth float,
                Labor_force_total int,
                Human_capital_index int,
                Population_grown_annual float
            );""")

# Month Table
cur.execute("""CREATE TABLE Month (
                    Month_Key int PRIMARY KEY,
                    Name varchar(255),
                    Quarter int,
                    Year int,
                    Decade int
                );""")

# Education Table
cur.execute("""CREATE TABLE Education (
                    Education_Key int PRIMARY KEY,
                    Total_Literacy_Rate float,
                    Male_Literacy_Rate float,
                    Female_Literacy_Rate float,
                    Primary_School_Enrollment float,
                    Post_Secondary_School_Enrollment float,
                    Public_Education_Spending float,
                    Pop_compuslory_school_age_total int,
                    Pop_offical_entrance_age_primary_total int,
                    Pop_offical_entrance_age_secondary_total int,
                    Teachers_primary_total int,
                    Teachers_secondary_total int
                );""")

# Health Table
cur.execute("""CREATE TABLE Health (
                    Health_Key int PRIMARY KEY,
                    Domestic_Health_Expenditure float,
                    Hospital_Beds float,
                    Immunization_attr_Hep int,
                    Immunization_attr_DPT int,
                    Immunization_attr_Measles int,
                    Immunization_attr_Polio int,
                    Num_Surgical_procedures int,
                    Num_Death_infant int,
                    Num_Death_stilbirths int,
                    Num_Death_Elderly int,
                    Num_health_professionals_Nurses float,
                    Num_health_professionals_Physicians float,
                    Prevalence_health_condition_overweight float,
                    Prevalence_health_condition_diabetes float,
                    Prevalence_health_condition_hiv float,
                    Adults_HIV_15up float,
                    Adults_new_HIV_15up float,
                    Children_HIV_under15 float,
                    Children_new_HIV_under15 float,
                    Homelessness_rate_male float,
                    Homelessness_rate_female float,
                    Homelessness_rate_total float,
                    CrimeRate float,
                    Cost_of_living_index float
                );""")

# Quality_of_Life Table
cur.execute("""CREATE TABLE Quality_of_Life(
                    Quality_of_Life_Key int PRIMARY KEY,
                    Access_to_Drinking_Water float,
                    Access_to_Sanitation float,
                    Access_to_Basic_Handwashing_Facilities float,
                    Unemployment_Rate_F float,
                    Unemployment_Rate_M float,
                    Unemployment_Rate_T float,
                    Access_to_Electricity_Total float,
                    Access_to_Electricity_Urban float,
                    Access_to_Electricity_Rural float,
                    Part_Time_Employment_T float, 
                    Part_Time_Employment_F float,    
                    Part_Time_Employment_M float
                );""")

# Population Table
cur.execute("""CREATE TABLE Population(
                    Population_Key int PRIMARY KEY,
                    Life_Expectancy_At_Birth_F float,
                    Life_Expectancy_At_Birth_M float,
                    Life_Expectancy_At_Birth_T float,
                    Net_Migration int,
                    Population_ages_0_15 int,
                    Population_ages_16_30 int,
                    Population_ages_31_64 int,
                    Population_ages_65_up int,
                    Rural_Population float,
                    Rural_Population_Growth_Rate float,
                    Rural_Poverty_Rate float,
                    Urban_Population float,
                    Urban_Population_Growth_Rate float,
                    Urban_Poverty_Rate float
                );""")

# Event Table
cur.execute("""CREATE TABLE Event (
                    Event_key int PRIMARY KEY,
                    Name varchar(255),
                    Disaster_Type varchar(255),
                    Start_Date Date,
                    End_Date Date,
                    Start_Month int,
                    End_Month int,
                    Disaster_Subgroup varchar(255),
                    Total_Deaths int,
                    No_Injured int,
                    No_Affected int
                );""")

# Fact Table
cur.execute("""CREATE TABLE Fact_Table (
                    Month_Key int,
                    Country_Key int,
                    Education_Key int,
                    Population_Key int,
                    Quality_of_Life_Key int,
                    Health_Key int,
                    Event_Key int,
                    Quality_of_life int,
                    Development_Index int,
                    Human_Development_Index int,
                    CONSTRAINT fk_month FOREIGN KEY(Month_Key) REFERENCES Month(Month_Key),
                    CONSTRAINT fk_country FOREIGN KEY(Country_Key) REFERENCES Country(Country_Key),
                    CONSTRAINT fk_education FOREIGN KEY(Education_Key) REFERENCES Education(Education_Key),
                    CONSTRAINT fk_population FOREIGN KEY(Population_Key) REFERENCES Population(Population_Key),
                    CONSTRAINT fk_quality_of_life FOREIGN KEY(Quality_of_Life_Key) REFERENCES Quality_Of_Life(Quality_of_Life_Key),
                    CONSTRAINT fk_health FOREIGN KEY(Health_Key) REFERENCES Health(Health_Key),
                    CONSTRAINT fk_event FOREIGN KEY(Event_Key) REFERENCES Event(Event_Key)
                );""")

cur.close()


In [40]:
cur = conn.cursor()

# cur.execute("DROP TABLE Fact_Table")
# cur.execute("DROP TABLE Country")
# cur.execute("DROP TABLE Month")
# cur.execute("DROP TABLE Education")
# cur.execute("DROP TABLE Health")
# cur.execute("DROP TABLE Quality_of_Life")
# cur.execute("DROP TABLE Population")
# cur.execute("DROP TABLE Event")

cur.close()


Rollback incase shit breaks

In [23]:
cur.execute("ROLLBACK")